THIS PROJECT HELPS IN FINDING LATITUDE AND LONGITUDE POSITIONS FOR THE GIVEN LIST OF POSTAL CODES

In [1]:
import pandas as pd
import requests
import numpy as np
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
!pip install geocoder
import geocoder

In [3]:
!pip install folium
import folium

In [4]:
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup

In [5]:
x= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text #extracting data from website
data = BeautifulSoup(x, 'lxml')

In [6]:
table_post = data.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_pc.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
df_pc['Borough'].replace('Not assigned', np.nan, inplace=True) #Removing Not assigned values
df_pc.dropna(subset=['Borough'], inplace=True)

df_pc.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df_pc2 = df_pc.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index() #grouping by Borough
df_pc2.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_pc2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df_pc2.groupby(['PostalCode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [11]:
df_2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Canada'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G') #Checking the function with M4G

[43.70941386000004, -79.36309957799995]

In [14]:
postal_codes = df_2['PostalCode']   #Obtaining latitude and longitude for all postal codes 
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist()]

In [16]:
df_2.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude']) #adding following attributes to dataframe
df_2['Latitude'] = df_coords['Latitude']
df_2['Longitude'] = df_coords['Longitude']

In [18]:
df_2.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


Now the Table contains Latitude and Longitude positions

# THANK YOU

Done By Saiharsha Reddy Pasula